## Import necessary packages

In [1]:
import scipy
from scipy import linalg as spla 
import numpy as np
from scipy.sparse import csr_matrix
import time

## Create sparse matrix and solution

In [2]:

# n = 5000
# M = scipy.sparse.rand( n, n, density = 0.1, random_state = 2025).tocsr()
# M.nnz/(n*n)

In [3]:
# download your matrix in Matrix Market Format in your current folder\data before running the cell
# https://math.nist.gov/MatrixMarket/data/misc/cylshell/s3rmt3m3.html
from scipy.io import mmread
M = (mmread('data\s3rmt3m3\s3rmt3m3.mtx')).tocsr()
n = M.shape[0]
# make it dense for comparison purpose used later
A = M.todense()
M

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 207695 stored elements and shape (5357, 5357)>

In [4]:
# create solution and RHS
x = np.array([1.0/i for i in range(1,n+1)])
b = M @ x

## Check the speed of dense solver and QR decomposition from numpy / scipy

In [5]:
%timeit np.linalg.solve(A, b)

882 ms ± 66.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit scipy.linalg.solve(A, b)

1.38 s ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit spla.qr(A)

7.97 s ± 297 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit np.linalg.qr(A)

9.99 s ± 221 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Choose numpy linear solver and scipy QR decomposition for speed

In [9]:
xsol = np.linalg.solve(A, b)
print(np.linalg.norm(xsol -x))

1.4921899639110952e-09


#### Check matrix reordering effect on memory for QR decomposition 

##### QR without reordering

In [10]:
sttime = time.time()
Q,R =spla.qr(A)
endtime = time.time()
print("Time for QR without reordering: ", endtime - sttime)

csr_matrix(Q).nnz,csr_matrix(R).nnz

Time for QR without reordering:  7.816354751586914


(16467165, 4133099)

In [11]:
xqr = spla.solve_triangular(R, Q.T @ b)
print(np.linalg.norm(xqr -x))

5.027933498591544e-09



##### QR with RCM ordering

In [12]:
sttime = time.time()
perm = scipy.sparse.csgraph.reverse_cuthill_mckee(M, symmetric_mode=True)
print(perm)
I,J = np.ix_(perm,perm)
bp = b[perm]
Mp = M[I, J]
t1 = time.time()
endtime = time.time()
time_reorder =  endtime - sttime
print("Time for reordering: ", endtime - sttime)


[   2    1    0 ... 5225 5224 5281]
Time for reordering:  0.006025791168212891


In [13]:
sttime = time.time()
Qp,Rp =spla.qr(Mp.todense())
endtime = time.time()
time_total = endtime - sttime + time_reorder
print("Time for QR with reordering: ", time_total)
csr_matrix(Qp).nnz,csr_matrix(Rp).nnz


Time for QR with reordering:  7.228511810302734


(14914709, 1233029)

In [14]:
xqrp = spla.solve_triangular(Rp, Qp.T @ bp)

print(np.linalg.norm(xqrp[np.argsort(perm)] -x))

6.15537933801292e-08


### memory saving

In [15]:

Qratio = csr_matrix(Qp).nnz / csr_matrix(Q).nnz
Rratio = csr_matrix(Rp).nnz / csr_matrix(R).nnz
print("Qratio: ", Qratio, "   Rratio: ", Rratio)

Qratio:  0.9057241486315343    Rratio:  0.29833038114983457
